In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install import-ipynb
%cd /content/drive/My\ Drive/Colab\ Notebooks/RetinaSmartCamera/notebooks

In [ ]:
#!pip install pyzmq
!pip install dippykit

     |████████████████████████████████| 61kB 2.7MB/s 
  Created wheel for dippykit: filename=dippykit-3.0.0-cp36-none-any.whl size=59000 sha256=14b432b6dfe23b8cbe9b155a6017a5486c9ecb496ef5722255c55f706dd798a0
  Stored in directory: /root/.cache/pip/wheels/69/1a/a7/80c66d6a60ad8f4bcd8eb783ff1cba9bbe7dfc4575e3b2f241
Successfully built dippykit


In [ ]:
import zmq
import time

context = zmq.Context()

#  Socket to talk to server
print("Connecting to hello world server…")
socket = context.socket(zmq.REQ)
socket.connect("tcp://localhost:5555")

#  Do 10 requests, waiting each time for a response
x = 2000000*'x'
y = 10000*'x'
start = time.time()
for request in range(10):
    print(f"Sending request {request} …")
    tracker = socket.send_string(x, copy=False, track=True)
    
    track_start = time.time()
    while tracker.done == False:
        pass
    track_end = time.time()

    #  Get the reply.
    message = socket.recv()
    #print(f"Received reply {request} [ {message} ]")
print(vars(tracker))
end = time.time()
print('Time taken %f'%(end-start))
#print('Time taken for one message %f'%(t2-t1))
print('Tracking time %f'%(track_end-track_start))

Connecting to hello world server…
Sending request 0 …
Sending request 1 …
Sending request 2 …
Sending request 3 …
Sending request 4 …
Sending request 5 …
Sending request 6 …
Sending request 7 …
Sending request 8 …
Sending request 9 …
{'events': {<threading.Event object at 0x103be7160>}, 'peers': set()}
Time taken 0.090617
Time taken for one message 0.001547
Tracking time 0.006302


In [ ]:
import import_ipynb
import RetinaUtils as ru

import numpy as np
from dippykit.coding import huffman_encode
from dippykit.coding import huffman_decode

import cv2
import re
import os
from sys import getsizeof

importing Jupyter notebook from RetinaUtils.ipynb


In [ ]:
video_path = '/content/drive/My Drive/Personal Dataset/Videos/Train/Gargoyle.MOV'

vidcap = cv2.VideoCapture(video_path)
success, img = vidcap.read()
count = 1
while success:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    

    success, img = vidcap.read()
    count += 1
print('Successfully converted video to %d frames'%count)

In [ ]:
import cv2
def imshow(img):
    x, y = img.shape[0], img.shape[1]
    plt.figure()
    plt.imshow(img, cmap='gray' if len(img.shape)==2 else None)
    plt.axis('off')

img_path = '/content/drive/My Drive/Personal Dataset/dock.jpg'
img_bgr = cv2.imread(img_path)
img = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
img_gray = cv2.imread(img_path, 0)

fixation = (1920/2, 1080/2)
data_dir = '/content/drive/My Drive/Colab Notebooks/RetinaSmartCamera/retina_data/'
Retina = ru.Retina(gpu=False)
Retina.loadLoc(data_dir+'ret8k_loc.pkl')
Retina.loadCoeff(data_dir+'ret8k_coeff.pkl')
Retina.prepare(img.shape, fixation)

Cortex = ru.Cortex(gpu=False)
cortex_dir = '/content/drive/My Drive/Colab Notebooks/RetinaSmartCamera/retina_data/cortex_data/'

Cortex.loadLocs(cortex_dir+'8k_cort_leftloc.pkl', cortex_dir+'8k_cort_rightloc.pkl')
Cortex.loadCoeffs(cortex_dir+'8k_cort_leftcoeff.pkl', cortex_dir+'8k_cort_rightcoeff.pkl')

In [ ]:
V = Retina.sample(img, fixation)
original_cort = Cortex.cort_img(V)
imshow(original_cort)
#imshow(original_cort.astype('uint32'))

flattened = V.reshape(-1)
print(V.dtype, V.shape)
seq, bits, code_dict, _ = huffman_encode(flattened.astype('uint32'))

In [ ]:
decoded = huffman_decode(seq, code_dict)
print(decoded.shape)
unflattened = decoded.reshape((8192, 3))
imshow(Cortex.cort_img(unflattened))